# MLlib

Spark biedt ook een framework aan voor MachineLearning modellen te trainen op gedistribueerde datasets.
Dit framework is MLlib of ook wel sparkML genoemd.
De code om te werken met deze package is sterk gelijkaardig aan sklearn.
De API en een uitgebreide documentatie met voorbeeldcode kan je [hier](https://spark.apache.org/docs/latest/ml-guide.html) vinden.

Deze package bied de volgende tools aan
* ML-technieken: classificatie, regressie, clustering, ...
* Features: Extracting en transforming van features, PCA, ...
* Pipelines: Maak, train, optimaliseer en evalueer pipelines
* Persistentie: Bewaar en laden van algoritmes/modellen
* Databeheer: Algebra tools, statistieken, null-waarden, ...

Let op dat er twee API's aangeboden worden, 1 gebaseerd op RDD's en 1 op DataFrames.
De API gebaseerd op RDD's is ouder en minder flexibel dan de API gebruik makend van DataFrames.
Momenteel werken ze allebei maar in de toekomst zou de RDD gebaseerde kunnen verdwijnen.

## Utilities

### Varianten voor numpy-arrays

Voor feature sets en volledige matrices van datasets aan te maken kan je gebruik maken van de Vector en Matrix klassen.
Deze beschikken over een Dense variant waar je elk element moet ingeven of een Sparse Variant waar cellen, elementen leeg kan laten.
Dit ziet er als volgt uit:

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MLLib intro").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 06:59:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.linalg import Vectors, Matrices

data = Vectors.sparse(4, (0, 1.0), (2, 1.0))
data = Vectors.dense([1.0, 0.0, 1.0, 0.0]) # zelfde vector als hierboven
data = Matrices.dense(4,4, range(16))
print(data)

DenseMatrix([[ 0.,  4.,  8., 12.],
             [ 1.,  5.,  9., 13.],
             [ 2.,  6., 10., 14.],
             [ 3.,  7., 11., 15.]])


Het is belangrijk om te weten dat dit locale datastructuren (wrapper rond numpy array) zijn en geen gedistribueerde objecten.

### Statistieken

Voor er kan gewerkt worden met statistieken moeten we (net zoals bij pandas) eerst een dataset hebben.
Hieronder maken we een random dataframe aan van 50 rijen en 4 kolommen.

In [7]:
from pyspark.mllib.random import RandomRDDs
sc = spark.sparkContext

# maak wat random data aan (50 rijen en 4 kolommen)
data = RandomRDDs.uniformVectorRDD(sc, 50, 4).map(lambda a: a.tolist()).toDF()
print(data.head(5))
data.summary().show()


[Row(_1=0.6923735011548775, _2=0.8566516404434346, _3=0.38898964799624125, _4=0.4146392116026095), Row(_1=0.3006456031525676, _2=0.6449957295745641, _3=0.8049665733988637, _4=0.7778958276334631), Row(_1=0.5961757789608558, _2=0.5627378050860062, _3=0.5640662118309633, _4=0.5611372562837672), Row(_1=0.9963077651043213, _2=0.6413155459143532, _3=0.8089928114531848, _4=0.028666595288168506), Row(_1=0.6068209371137967, _2=0.9933846956559048, _3=0.7214361528181714, _4=0.3855334877022376)]
+-------+-------------------+--------------------+--------------------+--------------------+
|summary|                 _1|                  _2|                  _3|                  _4|
+-------+-------------------+--------------------+--------------------+--------------------+
|  count|                 50|                  50|                  50|                  50|
|   mean| 0.5650113183106451|   0.513957360985025|  0.5113038509764116|  0.4469958822649707|
| stddev| 0.2667521082280384|  0.2785738381112

**Correlation matrix**

Buiten de statistieken die berekend kunnen worden door de summary() functie kan ook de correlatiematrix belangrijk zijn.
Deze matrix maakt het mogelijk om het verband tussen de verscheidene features te bestuderen.
Deze matrix kan als volgt berekend worden voor een gedistribueerd dataframe.

In [10]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=data.columns, outputCol='vector')
df = assembler.transform(data)
df.show()

df_corr = Correlation.corr(df, 'vector')
df_corr.show() # lastig om te interpreteren
df_corr.collect()[0][0].values

+-------------------+--------------------+-------------------+--------------------+--------------------+
|                 _1|                  _2|                 _3|                  _4|              vector|
+-------------------+--------------------+-------------------+--------------------+--------------------+
| 0.6923735011548775|  0.8566516404434346|0.38898964799624125|  0.4146392116026095|[0.69237350115487...|
| 0.3006456031525676|  0.6449957295745641| 0.8049665733988637|  0.7778958276334631|[0.30064560315256...|
| 0.5961757789608558|  0.5627378050860062| 0.5640662118309633|  0.5611372562837672|[0.59617577896085...|
| 0.9963077651043213|  0.6413155459143532| 0.8089928114531848|0.028666595288168506|[0.99630776510432...|
| 0.6068209371137967|  0.9933846956559048| 0.7214361528181714|  0.3855334877022376|[0.60682093711379...|
| 0.4745116448116551|  0.4829454889389445| 0.7488223144052799|  0.3076219786488289|[0.47451164481165...|
|0.09178104049249336|  0.1217772461580291| 0.5830332244

array([ 1.        , -0.01763239,  0.21089861, -0.21264896, -0.01763239,
        1.        , -0.05987249, -0.04682253,  0.21089861, -0.05987249,
        1.        ,  0.09710416, -0.21264896, -0.04682253,  0.09710416,
        1.        ])

**Onafhankelijksheidtest**

Naast de correlatiematrix kan het ook belangrijk zijn om de onafhankelijkheid te testen tussen elke feature en een label.
Dit kan uitgevoerd worden door een zogenaamde ChiSquareTest.
Deze krijgt als input een dataframe, de naam van de kolom met de features (als vectors) en de naam van een kolom met de labels.
We kunnen deze test uitvoeren als volgt:

In [14]:
import pyspark.sql.functions as f
from pyspark.ml.stat import ChiSquareTest

df = df.withColumn('label', f.when(f.rand() > 0.5, 1).otherwise(0))

ChiSquareTest.test(df, 'vector', 'label', flatten=True).show()


+------------+-------------------+----------------+-----------------+
|featureIndex|             pValue|degreesOfFreedom|        statistic|
+------------+-------------------+----------------+-----------------+
|           0| 0.4334366972557635|              49|50.00000000000005|
|           1|0.43343669725575984|              49|50.00000000000004|
|           2|0.43343669725575984|              49|50.00000000000004|
|           3|0.43343669725575984|              49|50.00000000000004|
+------------+-------------------+----------------+-----------------+



De Chi-square test wordt op de volgende manieren gebruikt in Machine Learning:
* Feature selectie: Identificeer welke features een significante relatie hebben met de targetvariabele.
* Correlatie tussen categorische variabelen: Evalueer de afhankelijkheid tussen twee categorische variabelen.

De Chi-square test vergelijkt de verwachte en werkelijke frequenties van waarden in de categorieën van een feature ten opzichte van de targetvariabele. Het doel is om te bepalen of de verschillen tussen deze frequenties statistisch significant zijn of niet. Een hoge Chi-square score betekent dat er een grote afhankelijkheid is tussen de feature en de target, wat suggereert dat de feature nuttig is voor voorspellingen.
Als de p-waarde kleiner is dan een vooraf bepaald significantieniveau (bijvoorbeeld 0,05), verwerp je de nullhypothese (dat er geen relatie is), wat betekent dat de feature relevant is.

**Summarizer**

Andere statistieken per kolom kunnen berekend worden door gebruik te maken van de Summarizer klasse:

In [19]:
from pyspark.ml.stat import Summarizer

summarizer = Summarizer.metrics('mean', 'count', 'max')
df.select(summarizer.summary(df.vector)).show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|aggregate_metrics(vector, 1.0)                                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|{[0.5650113183106451,0.513957360985025,0.5113038509764116,0.44699588226497067], 50, [0.9963077651043213,0.9933846956559048,0.988416086355417,0.9848219428953131]}|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+



Het gebruik maken van de Summarizer maakt het dus mogelijk om rechtstreeks op de feature vectors te werken zonder ze eerst terug te moeten splitsen.

### Pipelines

Pipelines binnen Spark zijn een groep van high-level API's steunend op Dataframes om ML-pipelines aan te maken, optimaliseren en trainen.
De belangrijkste concepten binnen de Pipelines van Spark zijn:
* Dataframe: concept van de dataset
* Transformer: Zet een dataframe om in een ander dataframe
* Estimator: Zet een dataframe om in een model/transformer
* Pipeline: een ketting van transformers en estimators om een flow vast te leggen
* Parameter: API voor parameters van transformers en estimators aan te passen

Gebruik nu onderstaande mini-dataset waar we op basis van een tekstkolom met logistische regressie een bepaald label proberen te voorspellen.
Maak hiervoor een Pipeline uit die bestaat uit de volgende stappen:
* Tokenizer om de tekstkolom te splitsen in de overeenkomstige woorden
* HashingTf om de term frequency van de woorden te bepalen en het om te zetten naar een feature vector
* LogisticRegression Estimator om de voorspelling te doen.

Train daarna deze pipeline en maak de voorspellingen voor de traningsdata.
Hoe accuraat is dit model?

In [29]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline

df = spark.createDataFrame([
    (0, 'dit is een stukje spark code', 1.0),
    (1, 'het is zo warm deze week pfff', 0.0),
    (2, 'spark is zo tof', 1.0),
    (3, 'ik maak veel oefeningen', 0.0)
], ['id', 'text', 'label'])

tokenizer = Tokenizer(inputCol='text', outputCol='tokens')
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol='features') # je kan de inputCol zelf typen of opvragen
logistic = LogisticRegression(maxIter=10, regParam=0.001) # featres default in features kolom, labels in label kolom

df_prep = hashingTF.transform(tokenizer.transform(df))
model = logistic.fit(df_prep)
model.transform(df_prep).show()

pipeline = Pipeline(stages=[tokenizer, hashingTF, logistic])
model = pipeline.fit(df)
predictions = model.transform(df)
predictions.show() # hier kan ik df gebruiken want de tokenizer en hashingTF zijn aanwezig in de pielinemodel

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
| id|                text|label|              tokens|            features|       rawPrediction|         probability|prediction|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|  0|dit is een stukje...|  1.0|[dit, is, een, st...|(262144,[53374,93...|[-7.0090024438898...|[9.02893686241666...|       1.0|
|  1|het is zo warm de...|  0.0|[het, is, zo, war...|(262144,[34996,10...|[6.43914122940283...|[0.99840477060621...|       0.0|
|  2|     spark is zo tof|  1.0|[spark, is, zo, tof]|(262144,[67406,10...|[-6.0958785077743...|[0.00224707006141...|       1.0|
|  3|ik maak veel oefe...|  0.0|[ik, maak, veel, ...|(262144,[67494,10...|[6.49357045919564...|[0.99848914936497...|       0.0|
+---+--------------------+-----+--------------------+--------------------+--------------------+---------

### Evalueren van een model

In de pyspark.ml package zitten er ook functionaliteiten voor deze modellen te evalueren.
Meer informatie hierover vind je [hier](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html).

In [30]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
evaluator.evaluate(predictions)

1.0

### Data sources

Door gebruik te maken van de sparkContext kunnen een reeks standaard databronnen ingelezen worden om datasets uit op te bouwen (Csv, Json, ...).
Daarnaast is het ook mogelijk om een folder met een reeks beelden te gebruiken als dataset om zo een model voor image classification te trainen.
Download nu [deze](https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin) dataset en upload ze naar een folder op het hadoop filesysteem.

In [31]:
import opendatasets as od

od.download("https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin")

Skipping, found downloaded files in "./chihuahua-or-muffin" (use force=True to force download)


In [32]:
from hdfs import InsecureClient

client = InsecureClient('http://localhost:9870', user='bigdata')
map = "/user/bigdata/MLLib"

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, 'chihuahua-or-muffin')

'/user/bigdata/MLLib/chihuahua-or-muffin'

De geuploade images kunnen nu ingelezen worden als volgt:

In [3]:
df = spark.read.format('image').option('dropInvalid', True).load('/user/bigdata/MLLib/chihuahua-or-muffin')
df.show()
df.select('image.origin', 'image.width').show(truncate=False)

+--------------------+
|               image|
+--------------------+
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
|{hdfs://namenode:...|
+--------------------+

+---------------------------------------------------------------------------+-----+
|origin                                                                     |width|
+---------------------------------------------------------------------------+-----+
|hdfs://namenode:9000/user/bigdata/MLLib/chihuahua-or-muffin/muffin-4.jpeg  |172  |
|hdfs://namenode:9000/user/bigdata/MLLib/chihuahua-or-muffin/muffin-7.jpeg  |171  |
|hdfs://namenode:9000/user/bigdata/MLLib/chihuahua-or-muffin/muffin-1.jpeg  |171  |
|hdfs://namenode:9000/user/bigdata/

Merk op dat het werken met images niet zo eenvoudig is.
Hiervoor wordt binnen pyspark typisch gebruik gemaakt van de [sparkdl](https://smurching.github.io/spark-deep-learning/site/api/python/sparkdl.html) package.
Hierbij staat de dl voor deep learning.
Aangezien dit ons momenteel te ver leidt ga ik dit niet verder toelichten.

Een andere aparte databron die eenvoudig ingelezen kan worden is het formaat "libsvm".
Een bestand van dit formaat wordt ingelezen als een dataframe met twee kolommen: een label en een kolom met de feature-vectors.
De code om dergelijk bestand in te laden is:

In [ ]:
spark.read.format('libsvm').load('{path to file}')

## Voorbeeld

Train en evalueer een machine learning model dat beeldgegevens kan classificeren, gebaseerd op eigenschappen zoals hoogte, breedte en aantal kanalen. Omdat MLlib geen directe ondersteuning biedt voor beelddata, zullen we enkele numerieke eigenschappen van de afbeeldingen gebruiken en pipelines opzetten voor preprocessing en training.
Hierbij ga je verder werken op het dataframe van de chichuahua of muffin dataset van hierboven.

Voer de volgende stappen uit:
* Data decoderen en omzetten naar een dataframe dat door ML-kan gebruikt worden.
    * Schrijf een udf om de image.data kolom om te zetten naar een vector. Gebruik hiervoor een udf dat dit eerst omzet naar een numpy array waarna het omgezet wordt naar een PIL-Image (een python package). Dit wordt dan geresized naar een 32x32 figuur. Ten slotte wordt dit omgezet naar een 1D-array 
    * Schrijf een udf om de filename in de image.origin kolom om te zetten naar 1 van de labels: 'muffin' of 'chihuahua'
* Na het uitvoeren van de udf's zou je een dataframe moeten hebben met minstens 2 kolommen: het label als string kolom, de data als een vector kolom
* Splits de data in een trainings en testset
* Voer de volgende preprocessing stappen uit
    * Zet de tekstuele labels om naar integers
    * Schaal de vector-kolom door elke waarde te delen door 255 zodat de pixel-waarden in de range 0-255 vallen
    * Zorg ervoor dat dit in een pipeline zit en fit de preprocessor al eens. Print het schema en een aantal rijen uit om de output te verifieren
* Voer data modelling uit
    * Zorg voor dimensionality reduction aan de hand van PCA
    * Gebruik logistische regressie om het label te voorspellen
* Evalueer het model door een confusion matrix te berekenen

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, DoubleType, StringType
from pyspark.ml.feature import StringIndexer, MinMaxScaler, PCA
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.functions import array_to_vector
from PIL import Image
import io
import numpy as np

target_size=(32,32)
num_channels=3
num_flattened=target_size[0]*target_size[1]*num_channels

def get_label(path):
    filename = path.split('/')[-1]
    if 'muffin' in filename:
        return 'muffin'
    else:
        return 'chihuahua'

def resize_image(pixel_data, width, height, target_size=(32, 32)):
    # Convert the pixel data to a numpy array and reshape it to (height, width)
    image_array = np.array(pixel_data, dtype=np.uint8).reshape((height, width, 3))
    # Convert the numpy array to a PIL Image
    image = Image.fromarray(image_array, mode="RGB")  # "L" for grayscale images
    # Resize the image to the target size
    image = image.resize(target_size)
    # image to list
    image = np.array(image, dtype=float).flatten().tolist()
    
    return image


df.printSchema()

# registreer de custom functions bij spark (maak udf's aan)
get_label_udf = F.udf(get_label, StringType()) # tweede parameter is het returntype
resize_image_udf = F.udf(resize_image, ArrayType(DoubleType())) # tweede parameter is het returntype

# gebruik de udf's
df_data = df.select('image.origin', array_to_vector(resize_image_udf(F.col('image.data'),F.col('image.width'),F.col('image.height'))).alias('data'))
df_data = df_data.withColumn('label', get_label_udf(F.col('origin')))
df_data.show()

# splits in train en testdata
train_data, test_data = df_data.randomSplit([0.8, 0.2], seed=15) # seed om niet random te werken maar altijd dezelfde splitings

# preprocessing
label_indexer = StringIndexer(inputCol='label', outputCol='label_indexed')
scaler = MinMaxScaler(inputCol='data', outputCol='scaled_features', max=255.0)
preprocessor = Pipeline(stages=[label_indexer, scaler])

preprocessor_model = preprocessor.fit(train_data)
train_data_pre = preprocessor_model.transform(train_data)
test_data_pre = preprocessor_model.transform(test_data)
train_data_pre.show()

# doe modelling
pca = PCA(k=5, inputCol='scaled_features', outputCol='pca_features')
lr = LogisticRegression(featuresCol='pca_features', labelCol='label_indexed', maxIter=10)
pipeline = Pipeline(stages=[preprocessor, pca, lr]) # je kan hier ook de nog niet gefitte pipeline gebruiken met de scaler als estimator, die wordt dan ook mee gefit

model = pipeline.fit(train_data)
predictions = model.transform(test_data)
predictions.show()

# evaluatie
preds_and_labels = predictions.select('prediction', 'label_index').rdd.map(lambda x: float(x[0]), float(x[1]))
metrics = MultiClassMetrics(preds_and_labels)
print(metrics.confusionMatrix().toArray())

print(metrics.accuracy)


root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

+--------------------+--------------------+---------+
|              origin|                data|    label|
+--------------------+--------------------+---------+
|hdfs://namenode:9...|[151.0,119.0,119....|   muffin|
|hdfs://namenode:9...|[254.0,254.0,254....|   muffin|
|hdfs://namenode:9...|[27.0,28.0,34.0,2...|   muffin|
|hdfs://namenode:9...|[24.0,99.0,155.0,...|   muffin|
|hdfs://namenode:9...|[193.0,174.0,168....|chihuahua|
|hdfs://namenode:9...|[168.0,159.0,177....|chihuahua|
|hdfs://namenode:9...|[204.0,191.0,223....|   muffin|
|hdfs://namenode:9...|[107.0,98.0,168.0...|chihuahua|
|hdfs://namenode:9...|[38.0,58.0,97.0,3...|   muffin|
|hdfs://namenode:9...|[204.0,212.0,228....|   muf

25/05/08 08:31:03 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


Py4JJavaError: An error occurred while calling o789.fit.
: java.lang.OutOfMemoryError: Java heap space
	at breeze.linalg.svd$.breeze$linalg$svd$$doSVD_Double(svd.scala:94)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:36)
	at breeze.linalg.svd$Svd_DM_Impl$.apply(svd.scala:35)
	at breeze.generic.UFunc.apply(UFunc.scala:47)
	at breeze.generic.UFunc.apply$(UFunc.scala:46)
	at breeze.linalg.svd$.apply(svd.scala:21)
	at org.apache.spark.mllib.linalg.distributed.RowMatrix.computePrincipalComponentsAndExplainedVariance(RowMatrix.scala:501)
	at org.apache.spark.mllib.feature.PCA.fit(PCA.scala:65)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:93)
	at org.apache.spark.ml.feature.PCA.fit(PCA.scala:64)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	a

## Oefening - classificatie

Voer de volgende stappen uit, volg de best-practices van het data science vak:
* Genereer een dataset met de make_classification functie van Scikit-Learn (of gebruik randomSplit op een bestaande dataset in PySpark).
* Converteer de dataset naar een PySpark DataFrame.
* Voer basisverkenning uit: toon het schema van de data, bekijk de eerste rijen, en controleer de kolomtypes.
* Gebruik VectorAssembler om alle features in één vector-kolom (features) te combineren.
* Controleer de structuur van de output.
* Train een Logistic Regression model op de gegenereerde dummy data.
* Gebruik een training/test split van 80/20.
* Evalueer het model door de nauwkeurigheid (accuracy) te berekenen.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.datasets import make_classification
import pandas as pd

# Maak dummy data
X, y = make_classification(n_samples=100, n_features=5, random_state=42)
df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(5)])
df['label'] = y

# Converteer naar Spark DataFrame
spark = SparkSession.builder.getOrCreate()
df_spark = spark.createDataFrame(df)

# Basisexploratie
df_spark.printSchema()
df_spark.show(5)

# Maak een assembler om features samen te voegen
assembler = VectorAssembler(inputCols=[f"feature_{i}" for i in range(5)], outputCol="features")
df_prepared = assembler.transform(df_spark)

# Bekijk het resultaat
df_prepared.select("features", "label").show(5, truncate=False)

# Train/test split
train_data, test_data = df_prepared.randomSplit([0.8, 0.2], seed=42)

# Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_data)

# Voorspellingen en evaluatie
predictions = model.transform(test_data)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy}")

## Oefening - hyperparameter tuning

Voer de volgende stappen uit, volg de best-practices van het data science vak:
* Genereer dummy data (of gebruik bestaande data) om een regressieprobleem op te lossen. (make_regression)
* Bouw een pipeline die de data voorbereidt door schaling uit te voeren, een RandomForest-model traint, en de beste hyperparameters selecteert. (Tip: ParamGridBuilder)
* Voer hyperparameter tuning uit en evalueer de prestaties van het beste model.

In [ ]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from sklearn.datasets import make_regression
import pandas as pd
from pyspark.sql import SparkSession

# Start een Spark sessie
spark = SparkSession.builder.getOrCreate()

# Genereer dummy regressie data
X, y = make_regression(n_samples=200, n_features=5, noise=0.1, random_state=42)
df = pd.DataFrame(X, columns=[f"feature_{i}" for i in range(5)])
df['label'] = y
df_spark = spark.createDataFrame(df)

# Stap 1: Data voorbereiden met VectorAssembler en MinMaxScaler
assembler = VectorAssembler(inputCols=[f"feature_{i}" for i in range(5)], outputCol="features_vector")
scaler = MinMaxScaler(inputCol="features_vector", outputCol="scaled_features")

# Stap 2: RandomForestRegressor model
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="label")

# Stap 3: Pipeline opbouwen
pipeline = Pipeline(stages=[assembler, scaler, rf])

# Stap 4: Hyperparameter grid instellen voor RandomForestRegressor
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 50]) \
    .addGrid(rf.maxDepth, [5, 10, 15]) \
    .build()

# Stap 5: CrossValidator instellen
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse"),
                          numFolds=3)  # 3-fold cross-validation

# Train/test split
train_data, test_data = df_spark.randomSplit([0.8, 0.2], seed=42)

# Stap 6: Model trainen en tunen
cv_model = crossval.fit(train_data)

# Stap 7: Het beste model toepassen op de test set
best_model = cv_model.bestModel
predictions = best_model.transform(test_data)

# Evaluatie van het beste model met RMSE (Root Mean Squared Error)
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Beste model RMSE op test data: {rmse}")

# Beste hyperparameters weergeven
print("Beste hyperparameters:")
print("Aantal bomen (numTrees):", best_model.stages[-1]._java_obj.getNumTrees())
print("Maximale diepte (maxDepth):", best_model.stages[-1]._java_obj.getMaxDepth())
